In [6]:
import nltk
import pandas as pd
from nltk.corpus import stopwords
import string
from nltk.tokenize import word_tokenize
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE
from nltk.corpus import webtext

In [4]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('webtext')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zhuko\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zhuko\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package webtext to
[nltk_data]     C:\Users\zhuko\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\webtext.zip.


True

In [7]:
# Подготовка данных
texts = webtext.sents()

In [8]:
texts

[['Cookie', 'Manager', ':', '"', 'Don', "'", 't', 'allow', 'sites', 'that', 'set', 'removed', 'cookies', 'to', 'set', 'future', 'cookies', '"', 'should', 'stay', 'checked', 'When', 'in', 'full', 'screen', 'mode', 'Pressing', 'Ctrl', '-', 'N', 'should', 'open', 'a', 'new', 'browser', 'when', 'only', 'download', 'dialog', 'is', 'left', 'open', 'add', 'icons', 'to', 'context', 'menu', 'So', 'called', '"', 'tab', 'bar', '"', 'should', 'be', 'made', 'a', 'proper', 'toolbar', 'or', 'given', 'the', 'ability', 'collapse', '/', 'expand', '.'], ['[', 'XUL', ']', 'Implement', 'Cocoa', '-', 'style', 'toolbar', 'customization', '.'], ...]

In [9]:
stop_words = set(stopwords.words('english'))

In [12]:
# Функция для предобработки текста
def preprocess_sentence(sentence):
    # Преобразование в нижний регистр
    sentence = [word.lower() for word in sentence]
    
    # Удаление знаков препинания и стоп-слов
    sentence = [word for word in sentence if word not in string.punctuation]
    sentence = [word for word in sentence if word not in stop_words]
    
    return sentence

In [13]:
def preprocess_corpus(corpus):
    return [preprocess_sentence(sentence) for sentence in corpus]

In [14]:
# Предобработка корпуса
texts = preprocess_corpus(texts)

In [96]:
n = 3
train_data, padded_sents = padded_everygram_pipeline(n, texts)

In [97]:
model = MLE(n) 
model.fit(train_data, padded_sents)

In [120]:
context = ("add","icons")

In [121]:
def generate_and_filter(model, num_words, context, random_seed):
    return list(filter(lambda w: w != '</s>', model.generate(num_words, context, random_seed)))

In [122]:
print("Следующие слова:", generate_and_filter(model, 5, context, random_seed=42))

Следующие слова: ['urls', 'listed', 'location', 'bar', 'search']


In [20]:
context_counts = model.context_counts(model.vocab.lookup(context))
sorted_words = sorted(context_counts.items(), key=lambda item: item[1], reverse=True)
top_5_words = [word for word, count in sorted_words[:5]]

In [123]:
context_counts = model.context_counts(model.vocab.lookup(context))
sorted_words = sorted(
    ((word, count) for word, count in context_counts.items() if word != '</s>'),
    key=lambda item: item[1],
    reverse=True)
top_5_words = [word for word, count in sorted_words[:5]]

In [125]:
print("Топ-5 следующих слов для заданного контекста:", top_5_words)

Топ-5 следующих слов для заданного контекста: ['context', 'urls']


In [127]:
# Проверка наличия контекста в корпусе
context_occurrences = sum(1 for sent in texts if context in zip(sent, sent[1:]))
print("Частота контекста в корпусе:", context_occurrences)

Частота контекста в корпусе: 2


In [129]:
context = ("coffee","shop")

In [130]:
def generate_and_filter(model, num_words, context, random_seed):
    return list(filter(lambda w: w != '</s>', model.generate(num_words, context, random_seed)))

In [132]:
print("Следующие слова:", generate_and_filter(model, 5, context, random_seed=42))

Следующие слова: ['counter', 'guy', 'lady', 'manhattan']


In [47]:
context_counts = model.context_counts(model.vocab.lookup(context))
sorted_words = sorted(
    ((word, count) for word, count in context_counts.items() if word != '</s>'),
    key=lambda item: item[1],
    reverse=True)
top_5_words = [word for word, count in sorted_words[:5]]

In [48]:
print("Топ-5 следующих слов для заданного контекста:", top_5_words)

Топ-5 следующих слов для заданного контекста: ['counter']


In [49]:
# Проверка наличия контекста в корпусе
context_occurrences = sum(1 for sent in texts if context in zip(sent, sent[1:]))
print("Частота контекста в корпусе:", context_occurrences)

Частота контекста в корпусе: 1


In [50]:
n = 2
train_data, padded_sents = padded_everygram_pipeline(n, texts)

In [51]:
model = MLE(n) 
model.fit(train_data, padded_sents)

In [94]:
context='restaurant'

In [90]:
def generate_and_filter(model, num_words, context, random_seed=42):
    return list(filter(lambda w: w != '</s>', model.generate(num_words, context, random_seed=42)))

In [95]:
print("Следующие слова:", generate_and_filter(model, 5, context))

Следующие слова: ['liquorice', 'always', 'easy', 'pick']


In [75]:
# Функция для получения топ-5 слов для заданного контекста
def get_top_5_next_words(model, context):
    # Получаем словарь вероятностей для всех слов, следующих за контекстом
    context = [context]
    candidates = model.context_counts(model.vocab.lookup(context))
    # Сортируем по вероятности и берем топ-5
    top_5 = sorted(((word, count) for word, count in candidates.items() if word != '</s>'), key=lambda item: item[1], reverse=True)[:5]
    return [word for word, count in top_5]

In [76]:
# Пример использования
 # пример слова-контекста
top_5_words = get_top_5_next_words(model, context)
print(f"Топ-5 слов после '{context}': {top_5_words}")

Топ-5 слов после 'restaurant': ['guy', 'half', 'talking', 'eat', 'everyone']
